In [1]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems

x, y, z  = -10..10, -10..10,  -10..10
box = x × y × z

using CairoMakie
using GLMakie
using ForwardDiff
using LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
@inbounds function Res(u, p, t)
    du1 = -u[2]-u[3]
    du2 = u[1]+p[1]*u[2]
    du3 = p[2]*u[1]-p[3]*u[3]+u[1]*u[3]
    return SVector(du1, du2, du3)
end
@inbounds function jac_res(u, p, t)
    SMatrix{3, 3}(0.0, 1.0, p[2]+u[3],
                -1.0, p[1], 0.0,
                -1.0, 0.0, -p[3]+u[1])
end

jac_res (generic function with 1 method)

In [3]:
time_set = (t = 500, Ttr = 250, tstep = 0.001)
integ_set = (alg = Vern9(), adaptive = false, dt = time_set.tstep)

(alg = Vern9(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false), lazy = true), adaptive = false, dt = 0.001)

In [86]:
 b = 0.3;  c = 4.89; a = 0.35;

In [87]:
u0 = SA[0.13060105886751308, 0.4782615699018184, 0.7435866656901606]
p = [a, b, c]
ds = CoupledODEs(Res, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  Res
 ODE solver:    Vern9
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [0.35, 0.3, 4.89]
 time:          0.0
 state:         [0.13060105886751308, 0.4782615699018184, 0.7435866656901606]


In [88]:
fp, ei, _ = fixedpoints(ds, box, jac_res)

(3-dimensional StateSpaceSet{Float64} with 1 points, Vector{ComplexF64}[[-4.8302785210864085 + 0.0im, 0.1451392605432047 - 0.9846627344351879im, 0.1451392605432047 + 0.9846627344351879im]], Bool[0])

In [89]:
Jacobian = jac_res(fp[1], p, 0)
eivecs = eigvecs(Jacobian)
vec_stable = real(eivecs[:, 1])
ϵ = 1e-23
shift = SA[0.0, 0.0, 0.0] + vec_stable*ϵ

3-element SVector{3, Float64} with indices SOneTo(3):
  1.9510202110332117e-24
 -3.7662457782753524e-25
  9.800595597386729e-24

In [90]:
Jacobi(u0, p, t) = ForwardDiff.jacobian((x) -> Res(x, p, 0), u0)

Jacobi (generic function with 1 method)

In [91]:
prob = ODEProblem(Res, shift, (0.0, 1000.0),p)

ODEProblem with uType SVector{3, Float64} and tType Float64. In-place: false
timespan: (0.0, 1000.0)
u0: 3-element SVector{3, Float64} with indices SOneTo(3):
  1.9510202110332117e-24
 -3.7662457782753524e-25
  9.800595597386729e-24

In [94]:
sol = solve(prob,alg = RK4(), adaptive = false, dt = 0.001);

In [95]:
ts, tf = 1, 900000
idx, idy, idz = 1, 2, 3
f = Figure(resolution = (1200, 800))
ax = LScene(f[1, 1])
lines!(ax, sol[idx, ts:tf], sol[idy, ts:tf], sol[idz, ts:tf], linewidth = 1.0, color = :deeppink)
#scatter!(fp[1][idx], fp[1][idy], fp[1][idz], color = :black)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)